### GAS process is too slow and not stable, so we recommend to process locally

In [1]:
import os

import cv2
import numpy as np
from PIL import Image
from tensorbay import GAS
from tensorbay.dataset import Dataset
from tqdm.notebook import tqdm
import traceback

def cat_to_one_hot(y_cat):
    cat2ind = {'Affection': 0, 'Anger': 1, 'Annoyance': 2, 'Anticipation': 3, 'Aversion': 4,
               'Confidence': 5, 'Disapproval': 6, 'Disconnection': 7, 'Disquietment': 8,
               'Doubt/Confusion': 9, 'Embarrassment': 10, 'Engagement': 11, 'Esteem': 12,
               'Excitement': 13, 'Fatigue': 14, 'Fear': 15, 'Happiness': 16, 'Pain': 17,
               'Peace': 18, 'Pleasure': 19, 'Sadness': 20, 'Sensitivity': 21, 'Suffering': 22,
               'Surprise': 23, 'Sympathy': 24, 'Yearning': 25}
    one_hot_cat = np.zeros(26)
    for em in y_cat:
        one_hot_cat[cat2ind[em]] = 1
    return one_hot_cat

gas = GAS('ACCESSKEY-05008025a6e33eb2cb23dddb819d123a')
dataset = Dataset("Emotic", gas)
segments = dataset.keys()
save_dir = './data/emotic_pre'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
try:
    for seg in ['test', 'val', 'train']:
        segment = dataset[seg]
        context_arr, body_arr, cat_arr, cont_arr = [], [], [], []-
        for data in tqdm(segment):
            with data.open() as fp:
                context = np.asarray(Image.open(fp))
            if len(context.shape) == 2:
                context = cv2.cvtColor(context, cv2.COLOR_GRAY2RGB)
            context_cv = cv2.resize(context, (224, 224))
            for label_box2d in data.label.box2d:
                xmin = label_box2d.xmin
                ymin = label_box2d.ymin
                xmax = label_box2d.xmax
                ymax = label_box2d.ymax
                body = context[ymin:ymax, xmin:xmax]
                body_cv = cv2.resize(body, (128, 128))
                context_arr.append(context_cv)
                body_arr.append(body_cv)
                cont_arr.append(np.array([int(label_box2d.attributes['valence']), int(label_box2d.attributes['arousal']), int(label_box2d.attributes['dominance'])]))
                cat_arr.append(np.array(cat_to_one_hot(label_box2d.attributes['categories'])))
        context_arr = np.array(context_arr)
        body_arr = np.array(body_arr)
        cat_arr = np.array(cat_arr)
        cont_arr = np.array(cont_arr)
        np.save(os.path.join(save_dir, '%s_context_arr.npy' % (seg)), context_arr)
        np.save(os.path.join(save_dir, '%s_body_arr.npy' % (seg)), body_arr)
        np.save(os.path.join(save_dir, '%s_cat_arr.npy' % (seg)), cat_arr)
        np.save(os.path.join(save_dir, '%s_cont_arr.npy' % (seg)), cont_arr) 
except Exception as e:
    print(traceback.format_exc())

  0%|          | 0/2088 [00:00<?, ?it/s]

C:\Users\BURN-E\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Traceback (most recent call last):
  File "C:\Users\BURN-E\AppData\Local\Temp/ipykernel_436/560295064.py", line 50, in <module>
    context_arr = np.array(context_arr)
ValueError: could not broadcast input array from shape (224,224,3) into shape (224,224)



In [2]:
def reshape_for_save(arr, common_shape, name):
    arr_copy = arr.copy()
    for i in range(len(arr_copy)):
        if arr_copy[i].shape != common_shape:
            print(i)
            arr_copy[i] = arr[i][:,:,:3]
    arr_copy = np.array(arr_copy)
    np.save(os.path.join(save_dir, f'{seg}_{name}_arr.npy'), arr_copy)

In [3]:
context_arr = reshape_for_save(context_arr,(224,224,3),'context')
body_arr = reshape_for_save(body_arr,(128,128,3),'body')

2550
2562
2563
2564
2631
2550
2562
2563
2564
2631


In [5]:
cat_arr = np.array(cat_arr)
cont_arr = np.array(cont_arr)
np.save(os.path.join(save_dir, '%s_cat_arr.npy' % (seg)), cat_arr)
np.save(os.path.join(save_dir, '%s_cont_arr.npy' % (seg)), cont_arr) 